In [69]:
# workspace = "dev"
# apiVersion = "5.2"
# moduleName = "Ed-Fi10"
# schoolYear = None
# districtId = None

StatementMeta(spark3p3med, 68, 26, Finished, Available)

In [70]:
%run OEA_py

StatementMeta(, 68, -1, Finished, Available)

2023-07-06 21:32:20,954 - OEA - INFO - Now using workspace: dev
2023-07-06 21:32:20,954 - OEA - INFO - OEA initialized.


In [71]:
oea.set_workspace(workspace)

StatementMeta(spark3p3med, 68, 28, Finished, Available)

2023-07-06 21:32:21,335 - OEA - INFO - Now using workspace: dev


In [72]:
districtPath = districtId if districtId != None else "All"
schoolYearPath = schoolYear if schoolYear != None else "All"
items = oea.get_folders(f'stage1/Transactional/{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/ed-fi')
print(items)

StatementMeta(spark3p3med, 68, 29, Finished, Available)

['absenceEventCategoryDescriptors', 'academicHonorCategoryDescriptors', 'academicSubjectDescriptors', 'academicWeeks', 'accommodationDescriptors', 'accountClassificationDescriptors', 'accountCodes', 'accountabilityRatings', 'accounts', 'achievementCategoryDescriptors', 'actuals', 'additionalCreditTypeDescriptors', 'addressTypeDescriptors', 'administrationEnvironmentDescriptors', 'administrativeFundingControlDescriptors', 'ancestryEthnicOriginDescriptors', 'assessmentCategoryDescriptors', 'assessmentIdentificationSystemDescriptors', 'assessmentItemCategoryDescriptors', 'assessmentItemResultDescriptors', 'assessmentItems', 'assessmentPeriodDescriptors', 'assessmentReportingMethodDescriptors', 'assessmentScoreRangeLearningStandards', 'assessments', 'attemptStatusDescriptors', 'attendanceEventCategoryDescriptors', 'behaviorDescriptors', 'bellSchedules', 'budgets', 'calendarDates', 'calendarEventDescriptors', 'calendarTypeDescriptors', 'calendars', 'careerPathwayDescriptors', 'charterApprov

In [73]:

def ingest_edfi_dataset(tables_source):
    full_source_path = f"stage1/Transactional/{tables_source}"
    items = oea.get_folders(full_source_path)
    options = {'header':False}
    for item in items: 
        table_path = full_source_path +'/'+ item
        try:
            entity_path = f"{tables_source}/{item}"
            print(entity_path)
            if item == 'metadata.csv':
                logger.info('ignore metadata processing, since this is not a table to be ingested')
            else:
                oea.ingest(entity_path, 'id', options)
        except AnalysisException as e:
            # This means the table may have not been properly refined due to errors with the primary key not aligning with columns expected in the lookup table.
            pass

StatementMeta(spark3p3med, 68, 30, Finished, Available)

In [74]:
ingest_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/ed-fi')
ingest_edfi_dataset(f'{moduleName}/{apiVersion}/DistrictId={districtPath}/SchoolYear={schoolYearPath}/tpdm')

StatementMeta(spark3p3med, 68, 31, Submitted, Running)

Ed-Fi10/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors
2023-07-06 21:32:23,124 - OEA - INFO - Ingesting from: stage1/Transactional/Ed-Fi10/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors, batch type of: delta, source data format of: json
source_path is: abfss://oea@stoeadablickw3.dfs.core.windows.net/dev/stage1/Transactional/Ed-Fi10/5.2/DistrictId=All/SchoolYear=All/ed-fi/absenceEventCategoryDescriptors/delta_batch_data
2023-07-06 21:32:25,000 - OEA - INFO - Number of new inbound rows processed: 0
Ed-Fi10/5.2/DistrictId=All/SchoolYear=All/ed-fi/academicHonorCategoryDescriptors
2023-07-06 21:32:25,122 - OEA - INFO - Ingesting from: stage1/Transactional/Ed-Fi10/5.2/DistrictId=All/SchoolYear=All/ed-fi/academicHonorCategoryDescriptors, batch type of: delta, source data format of: json
source_path is: abfss://oea@stoeadablickw3.dfs.core.windows.net/dev/stage1/Transactional/Ed-Fi10/5.2/DistrictId=All/SchoolYear=All/ed-fi/academicHonorCategoryD